In [117]:
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

In [118]:
def random_dates(start, end, n=10):

    start_u = start.value//10**9
    end_u = end.value//10**9

    return pd.to_datetime(np.random.randint(start_u, end_u, n), unit='s')

In [50]:
clientesdf = pd.DataFrame(columns = ['id_pager', 'numero_telefono', 'codigo_de_area', 'fecha_creacion_cuenta', 'nombre_cliente', 'region', 'categoria_cliente'])
eventosdf = pd.DataFrame(columns = ['hora', 'minutos', 'segundos', 'id_pager', 'mensaje', 'numero_original'])

start_10 = pd.to_datetime('1997-1-1')
end_10 = pd.to_datetime('2003-1-7')

start = pd.to_datetime('1997-1-1')
end = pd.to_datetime('2019-11-16')

nombre = ['Aiko', 'Chiyoko', 'Fuji', 'Fumiko', 'Haru', 'Hatsu', 'Isamu', 'Kameyo', 'Keiko', 'Kimi', 'Ichiro', 'Jomei', 'Kaori', 'Kichiro']
regiones = ['hokkaido', 'okinawa', 'kyushu', 'chugoku', 'shikoku', 'kinki', 'chubu', 'kanto', 'tohoku']
categorias = ['hierro','hierro','hierro','oro', 'oro', 'platino', 'basico', 'basico', 'basico', 'basico']

for _ in range(500): # En realidad, segun ejercicio son 500 !
    fecha = random_dates(start_10, end_10, 1)[0]
    fecha = pd.Timestamp(fecha.year, fecha.month, fecha.day)
    clientesdf = clientesdf.append({
                    'id_pager' : _,  
                    'numero_telefono' : random.randint(100000000,200000000),
                    'codigo_de_area' : random.randint(100,700),
                    'fecha_creacion_cuenta' : fecha, #random_dates(start_10, end_10, 1)[0],
                    'nombre_cliente' : random.choice(nombre), 
                    'region' : random.choice(regiones),
                    'categoria_cliente' : random.choice(categorias)
                }, ignore_index = True)
for i in range(10000):
    eventosdf = eventosdf.append({
                    'year' : random.randint(1997,2019), 
                    'month' : random.randint(1,12), 
                    'day' : random.randint(1,28),
                    'hora' : random.randint(1,24),
                    'minutos' : random.randint(1,60),
                    'segundos' : random.randint(1,60),
                    'id_pager' : random.randint(0,499),
                    'mensaje' : 'Mensaje',
                    'numero_original' : random.randint(100000000,200000000)
                }, ignore_index = True)

### Ejercicio 2 Pandas
Tokyo Telemessage se encuentra analizando la posibilidad de dar finalmente de baja su servicio de pager que mantienen desde la decada de 1980. Los pagers son un dispositivo de mensageria que realiza un 'beep' al recibir un mensaje de texto de un numero limitado de caracteres. Para poder enviar un mensaje, uno debe llamar a un numero telefonico que representa al pager dejando un mensaje a una operadora.

Para el analisis se cuenta con dos csv's: clientes.csv y eventos.csv

Como parte del analisis se debe responder:
- a ¿Cual es la region que aun tiene activos la mayor cantidad de pagers, entendiendo como activos aquellos que recibieron por lo menos un mensage en el ultimo més?
- b ¿Cúal es el porcentaje de pagers activos que solamente reciben mensajes unicamente de una persona (es decir, siempre desde el mismo numero)?

Utilizar pandas para responder las anteriores preguntas.

a

In [51]:
eventosdf["fecha"] = pd.to_datetime(eventosdf[['year', 'month', 'day']])

In [111]:
eventosAct = eventosdf[eventosdf['fecha'] >= (pd.to_datetime('today') - pd.DateOffset(months=1))]
eventosAct = eventosAct.drop_duplicates('id_pager')

In [112]:
pagersAct = eventosAct.merge(clientesdf, on='id_pager', how='inner')

In [114]:
grup = pagersAct.groupby('region')['mensaje'].agg(len).reset_index()
grup.rename(columns={'mensaje':'total'}).nlargest(1,'total')

,region,total
0,chubu,11


b

In [169]:
eventosAct = eventosdf[eventosdf['fecha'] >= (pd.to_datetime('today') - pd.DateOffset(months=1))]
eventosActUnicos = eventosAct.drop_duplicates('id_pager')

activos = eventosActUnicos['id_pager'].count()

pagersAct = eventosAct.merge(clientesdf, on='id_pager', how='inner')

In [170]:
test= pagersAct.groupby(['id_pager'])['numero_original'].agg({"numero_original": pd.Series.nunique}).reset_index()
test = test.loc[test['numero_original'] == 1]
unicos = test['id_pager'].count()

/home/giaco/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


In [171]:
porciento = (unicos/activos)*100
porciento

86.8421052631579

# Parcial 2018

El equipo comercial en Argentina de instagram cuenta con informacion sumarizada de usuarios del país en el siguiente Data Frame (id_usuario, nickname, pais, fecha_alta, catidad_seguidores, cantidad_seguidos, cantidad_total_publicaciones).

Uno de sus analistas quiere obtener los potenciales 10 usuarios "influencers" que van a liderar su primera campaña de via publica en el paiel criterio final para seleccionar a los candidatos será a partir de la mayor cantidad de seguidores que tengan.s y para ello necesita de nuestra ayuda. El criterio para detectar un influences es que el mismo debe:
- Tener una cantidad de seguidores mayor al 80% de la máxima cantidad de seguidores de todos los usuarios dentro de su pais.(Por ejemplo, si un usuario es Argentino Y el máximo de seguidores en Argentina es 1000 entonces requerira tener al menos 800 seguidores).
- El numero de usuarios que esa persona sigue no debe superar el 15% de lacantidad de seguidores que el tiene.
- El criterio final para seleccionar a los candidatos será a partir de la mayor cantidad de seguidores que tengan.

In [4]:
def random_dates(start, end, n=10):

    start_u = start.value//10**9
    end_u = end.value//10**9

    return pd.to_datetime(np.random.randint(start_u, end_u, n), unit='s')

instagram = pd.DataFrame(columns = ['id_usuario', 'nickname', 'pais', 'fecha_alta', 'catidad_seguidores', 'cantidad_seguidos', 'cantidad_total_publicaciones'])

start = pd.to_datetime('2010-10-6')
end = pd.to_datetime('today')

pais = ['Argentina', 'Bolivia', 'Chile', 'Gabón', 'Kenia', 'Estados Unidos', 'España', 'Japón', 'Lituania']
nickname = ['Abby','Addie','Aggie','Big Ed', 'Bitsy', 'Blondy', 'Brooksie', 'Buddy', 'Chappie', 'Cliffy','Aiko', 'Chiyoko', 'Fuji', 'Fumiko', 'Haru', 'Hatsu', 'Isamu', 'Kameyo', 'Keiko', 'Kimi', 'Ichiro', 'Jomei', 'Kaori', 'Kichiro']

for i in range(5000): # En realidad, segun ejercicio son 500 !
    fecha = random_dates(start, end, 1)[0]
    fecha = pd.Timestamp(fecha.year, fecha.month, fecha.day)
    instagram = instagram.append({
                    'id_usuario' : i,  
                    'nickname' : random.choice(nickname),
                    'pais' : random.choice(pais),
                    'fecha_alta' : fecha, #random_dates(start_10, end_10, 1)[0],
                    'catidad_seguidores' : random.randint(0,1000), 
                    'cantidad_seguidos' : random.randint(0,1000),
                    'cantidad_total_publicaciones' : random.randint(0,150)
                }, ignore_index = True)

In [5]:
instagram.head()

,id_usuario,nickname,pais,fecha_alta,catidad_seguidores,cantidad_seguidos,cantidad_total_publicaciones
0,0,Aggie,Lituania,2017-08-18,869,433,79
1,1,Kaori,Bolivia,2012-09-19,727,202,121
2,2,Chiyoko,Bolivia,2019-10-19,836,284,77
3,3,Kameyo,Kenia,2010-11-22,742,814,143
4,4,Fuji,Kenia,2013-12-13,447,204,80


In [11]:
igArge = instagram.loc[instagram['pais']=='Argentina']
maxSeguidoresArgentina = igArge['catidad_seguidores'].max()
influencers = igArge.loc[igArge['catidad_seguidores'] >= maxSeguidoresArgentina*0.8]
influencers = influencers.loc[igArge['catidad_seguidores'] >= igArge['cantidad_seguidos']*1.15]
influencers['catidad_seguidores'] = influencers['catidad_seguidores'].astype(int)
influencers.nlargest(10, 'catidad_seguidores')

,id_usuario,nickname,pais,fecha_alta,catidad_seguidores,cantidad_seguidos,cantidad_total_publicaciones
2230,2230,Aggie,Argentina,2011-11-06,1000,386,113
4016,4016,Aiko,Argentina,2018-01-09,999,585,140
2954,2954,Keiko,Argentina,2015-11-22,998,829,73
917,917,Fumiko,Argentina,2013-03-06,997,695,52
1177,1177,Abby,Argentina,2013-04-23,997,707,51
3478,3478,Fumiko,Argentina,2018-12-03,997,699,28
766,766,Cliffy,Argentina,2018-04-16,992,337,129
3030,3030,Cliffy,Argentina,2012-05-05,991,318,11
1539,1539,Big Ed,Argentina,2019-07-14,988,449,150
1828,1828,Bitsy,Argentina,2015-11-20,985,778,6


La empreza instagram almacena informacion de usuarios en un RDD de la forma (id_usuario, nickname, pais, fecha_alta) y en otro RDD almacena informacion de las publicaciones como (id_publicacion, id_usuario, id_foto, comentario, fecha). En Instagram, un hashtag es una palabra que comienza con #. Se pide realizar lo siguiente utilizando el api de RDD en pyspark:

- a. Encontrar el top 10 de hashtags que más aparecen y que fueron publicados únicamente por usuarios del país Argentina.(Por ejemplo: Si hashtag fue publicado por algien que no es de Argentina entonces no hay que considerar dicho hastag para el top 10, aunque sea el que mayor cantidad de ocurncias haya acumulado).
- b. Construr y cachear un RDD con los pares de usuarios (id_usuario1, id_usuario2) que tienen hastags publicados en comun.

In [29]:
#id_usuario, nick pais fecha_alta 
jugadordd = [
    (1,"Mario Kart", "argentina",2019),
    (2,"Mario","argentina",2019),
    (3,"gon", "no",2019),
    (4,"pan", "argentina",2019),
    (5,"win",  "argentina",2019),
    (6,"nop", "argentina",2019),
    (7,"capaz", "no",2019),
    (11,"Mario Kart", "argentina",2019),
    (21,"Mario","argentina",2019),
    (31,"gon", "no",2019),
    (41,"pan", "argentina",2019),
    (51,"win",  "argentina",2019),
    (61,"nop", "argentina",2019),
    (71,"capaz", "no",2019),
    (12,"Mario Kart", "argentina",2019),
    (22,"Mario","argentina",2019),
    (32,"gon", "no",2019),
    (42,"pan", "argentina",2019),
    (52,"win",  "argentina",2019),
    (62,"nop", "argentina",2019),
    (72,"capaz", "no",2019),
    (13,"Mario Kart", "argentina",2019),
    (23,"Mario","argentina",2019),
    (33,"gon", "no",2019),
    (43,"pan", "argentina",2019),
    (53,"win",  "argentina",2019),
    (63,"nop", "argentina",2019),
    (73,"capaz", "no",2019)
]
#id_publicacion , id_usuario, id_foto, comentario, fecha
estadisticardd = [
    (21,1, 15, "hola #lindo",1),
    (21,3, 15, "hola #lindo",1),
    (21,4, 15, "hola #lindo",1),
    (21,5, 15,"hola #lindo",1),
    (21,6, 15, "hola ",1),
    (21,7, 15, "#1",1),
    (22,1, 4.1,"#noooo",1),
    (22,2, 4.1,"#bff",1),
    (22,5, 4.1,"sos lo mas #amigos",1),
    (23,1, 128, "sos lo mas #amigos",1),
    (24,1, 43, "sos lo mas #amigos",1),
    (25,2, 32.5, "sos #amigos ehse",1),
    (26,3, 58, "jajjajaj",1),
    (26,4, 58, ":)",1),
    (26,6, 58, ":)",1),
    (26,5, 58, ":)",1),
    (27,4, 25, ":)",1),
    (21,12, 15, "hola #lindo",1),
    (21,31, 15, "hola #lindo",1),
    (21,41, 15, "hola #lindo",1),
    (21,53, 15,"hola #lindo",1),
    (21,61, 15, "hola ",1),
    (21,72, 15, "#1",1),
    (22,13, 4.1,"#noooo",1),
    (22,22, 4.1,"#bff",1),
    (22,51, 4.1,"sos lo mas #amigos",1),
    (23,11, 128, "sos lo mas #amigos",1),
    (24,12, 43, "sos lo mas #amigos",1),
    (25,23, 32.5, "sos #amigos ehse",1),
    (26,31, 58, "jajjajaj",1),
    (21,12, 15, "hola #lindo",1),
    (21,31, 15, "hola #lindo",1),
    (21,41, 15, "hola #lindo",1),
    (21,51, 15,"hola #lindo",1),
    (21,62, 15, "hola ",1),
    (21,72, 15, "#1",1),
    (22,12, 4.1,"#noooo",1),
    (22,22, 4.1,"#bff",1),
    (22,53, 4.1,"sos lo mas #amigos",1),
    (23,11, 128, "sos lo mas #amigos",1),
    (24,12, 43, "sos lo mas #amigos",1),
    (25,22, 32.5, "sos #amigos ehse",1),
    (26,33, 58, "jajjajaj",1),
    (21,41, 15, "hola #lindo",1),
    (21,53, 15,"hola #lindo",1),
    (21,61, 15, "hola ",1),
    (21,72, 15, "#1",1),
    (22,13, 4.1,"#noooo",1),
    (22,22, 4.1,"#bff",1),
    (22,51, 4.1,"sos lo mas #amigos",1),
    (23,11, 128, "sos lo mas #amigos",1),
    (24,12, 43, "sos lo mas #amigos",1),
    (25,23, 32.5, "sos #amigos ehse",1),
    (26,31, 58, "jajjajaj",1),
    (21,12, 15, "hola #lindo",1),
    (21,31, 15, "hola #lindo",1),
    (21,41, 15, "hola #lindo",1),
    (21,51, 15,"hola #lindo",1),
    (21,62, 15, "hola ",1),
    (21,72, 15, "#1",1),
    (22,12, 4.1,"#noooo",1),
    (22,22, 4.1,"#bff",1),
    (22,53, 4.1,"sos lo mas #amigos",1),
    (23,11, 128, "sos lo mas #amigos",1),
    (24,12, 43, "sos lo mas #amigos",1),
    (25,22, 32.5, "sos #amigos ehse",1),
    (26,33, 58, "jajjajaj",1)
]

usuario = sc.parallelize(jugadordd)
publicacion = sc.parallelize(estadisticardd)

In [69]:
usuariosNoArg = usuario.filter(lambda x: x[2] != 'argentina')
usuariosNoArg = usuariosNoArg.map(lambda x: (x[0], x[1]))

publi = publicacion.map(lambda x: (x[1],x[3]))
publi = publi.join(usuariosNoArg)

palabritas = publi.flatMap(lambda x: x[1][0].split())
hashtages = palabritas.filter(lambda x: x.startswith("#"))
hashtages = hashtages.map(lambda x: (x,1))
hashtagesNoArg = hashtages.reduceByKey(lambda x,y: x+y)
hashnoArg = hashtagesNoArg.map(lambda x: x[0])

In [70]:
usuariosArg = usuario.filter(lambda x: x[2] == 'argentina')
usuariosArg = usuariosArg.map(lambda x: (x[0], x[1]))

publi = publicacion.map(lambda x: (x[1],x[3]))
publi = publi.join(usuariosArg)

palabritas = publi.flatMap(lambda x: x[1][0].split())
hashtages = palabritas.filter(lambda x: x.startswith("#"))
hashtages = hashtages.map(lambda x: (x,1))
hashtagesArg = hashtages.reduceByKey(lambda x,y: y+x)
hashtagesArg.collect()

[('#bff', 5), ('#noooo', 5), ('#lindo', 14), ('#amigos', 20)]

In [80]:
hashtagesArg.leftOuterJoin(hashtagesNoArg)\
    .filter(lambda x: x[1][1]==None)\
    .map(lambda x: (x[0],x[1][0]))\
    .takeOrdered(10, key = lambda x: x[0])

[('#amigos', 20), ('#bff', 5), ('#noooo', 5)]

b

In [92]:
publi = publicacion.map(lambda x: (x[1],x[3]))
hashtages = publi.flatMap(lambda x: ((x[0], y) for y in  x[1].split()))
hashtages = hashtages.filter(lambda x: x[1].startswith("#"))
rdd = hashtages.map(lambda x: (x[1],x[0]))
rdd = rdd.join(rdd)
rdd = rdd.cache()
rdd.collect()

[('#bff', (2, 2)),
 ('#bff', (2, 22)),
 ('#bff', (2, 22)),
 ('#bff', (2, 22)),
 ('#bff', (2, 22)),
 ('#bff', (22, 2)),
 ('#bff', (22, 22)),
 ('#bff', (22, 22)),
 ('#bff', (22, 22)),
 ('#bff', (22, 22)),
 ('#bff', (22, 2)),
 ('#bff', (22, 22)),
 ('#bff', (22, 22)),
 ('#bff', (22, 22)),
 ('#bff', (22, 22)),
 ('#bff', (22, 2)),
 ('#bff', (22, 22)),
 ('#bff', (22, 22)),
 ('#bff', (22, 22)),
 ('#bff', (22, 22)),
 ('#bff', (22, 2)),
 ('#bff', (22, 22)),
 ('#bff', (22, 22)),
 ('#bff', (22, 22)),
 ('#bff', (22, 22)),
 ('#1', (7, 7)),
 ('#1', (7, 72)),
 ('#1', (7, 72)),
 ('#1', (7, 72)),
 ('#1', (7, 72)),
 ('#1', (72, 7)),
 ('#1', (72, 72)),
 ('#1', (72, 72)),
 ('#1', (72, 72)),
 ('#1', (72, 72)),
 ('#1', (72, 7)),
 ('#1', (72, 72)),
 ('#1', (72, 72)),
 ('#1', (72, 72)),
 ('#1', (72, 72)),
 ('#1', (72, 7)),
 ('#1', (72, 72)),
 ('#1', (72, 72)),
 ('#1', (72, 72)),
 ('#1', (72, 72)),
 ('#1', (72, 7)),
 ('#1', (72, 72)),
 ('#1', (72, 72)),
 ('#1', (72, 72)),
 ('#1', (72, 72)),
 ('#noooo', (1, 1)),

# Parcial 2019

ACARA posee información histórica sobre la venta de autos 0 km en la Argentina. Posee un RDD con información de cada modelo (marca, modelo, motor, transmisión, origen) y otro con la información de ventas (marca, modelo, fecha, concesionario).

Se desea conocer, para los modelos de origen nacional, cuales son los modelos que ya se discontinuaron(un modelo discontinuado es aquel que no tuvo ventas en los últimos 12 meses), obteniendo un nuevoRDD con (marca, modelo, total_vendido, fecha_inicio_venta, fecha_discontinuación),donde la fecha de discontinuación es la fecha cuando se vendió el último autode ese modelo, ordenado ascendentemente por esta fecha.

*Aclaración:Se puede asumir que el primer RDD tiene un único registro para cada Marca y Modelo.*

In [98]:
modelo = [
    ('Chevrolet','Aveo', 'Gasolina', 'Manual', 'Estados Unidos'),
    ('Chevrolet','Cobalt', 'Híbridos', 'Automatico', 'Estados Unidos'),
    ('Chevrolet','Cruze', 'Gasolina', 'Manual', 'Argentina'),
    ('Fiat','Uno', 'Gasolina', 'Automatico', 'Argentina'),
    ('Fiat','500', 'Gasolina', 'Manual', 'Italia'),
    ('Fiat','Toro', 'Diecel', 'Manual', 'Argentina'),
    ('Tesla','S', 'Electrico', 'Automatico', 'Estados Unidos'),
    ('Tesla','E', 'Electrico', 'Automatico', 'Estados Unidos'),
    ('Tesla','X', 'Electrico', 'Automatico', 'Argentina'),
    ('Tesla','Y', 'Electrico', 'Automatico', 'Estados Unidos'),
    ('Audi','A7', 'Diecel', 'Manual', 'Alemania'),
    ('Audi','A8', 'Diecel', 'Manual', 'Argentina'),
    ('Audi','R8', 'Diecel', 'Manual', 'Alemania'),
    ('Audi','Q8', 'Diecel', 'Manual', 'Argentina'),
    ('BMW','Coupé', 'Diecel', 'Manual', 'Argentina'),
    ('BMW','Cabriolet', 'Diecel', 'Manual', 'Alemania'),
    ('BMW','Sedan', 'Híbridos', 'Automatico', 'Alemania'),
    ('BMW','X4', 'Diecel', 'Manual', 'Alemania')
]

ventas = [
    ('Chevrolet','Aveo', '2001-06-05', 'Sara'),
    ('Chevrolet','Cobalt', '2010-11-19', 'Sara'),
    ('Chevrolet','Cruze', '2011-05-05', 'Beta Automotores'),
    ('Fiat','Uno', '2019-11-05', 'Sara'),
    ('Fiat','500', '2019-06-05', 'Beta Automotores'),
    ('Fiat','Toro', '2001-06-05', 'Sara'),
    ('Tesla','S', '2019-01-10', 'Sara'),
    ('Tesla','E', '2019-09-18', 'Beta Automotores'),
    ('Tesla','X', '2018-06-05', 'Beta Automotores'),
    ('Tesla','Y', '2016-06-05', 'Beta Automotores'),
    ('Audi','A7', '2011-06-05', 'Sara'),
    ('Audi','A8', '2002-06-05', 'DI CHIARA CUMMINS'),
    ('Audi','R8', '2019-06-05', 'DI CHIARA CUMMINS'),
    ('Audi','Q8', '2018-06-05', 'Sara'),
    ('BMW','Coupé', '2019-06-05', 'DI CHIARA CUMMINS'),
    ('BMW','Cabriolet', '2019-06-05', 'Sara'),
    ('BMW','Sedan', '2018-06-05', 'DI CHIARA CUMMINS'),
    ('BMW','X4', '2019-06-05', 'DI CHIARA CUMMINS'),
    ('Fiat','Uno', '2013-06-05', 'Sara'),
    ('Fiat','500', '2019-10-05', 'DI CHIARA CUMMINS'),
    ('Fiat','Toro', '2019-11-05', 'Sara'),
    ('Tesla','S', '2001-11-05', 'Sara'),
    ('Tesla','E', '2019-11-05', 'DI CHIARA CUMMINS'),
    ('Tesla','X', '2001-06-05', 'Sara'),
    ('Tesla','Y', '2019-09-05', 'Sara'),
    ('Audi','A7', '2001-06-05', 'Sara'),
    ('Audi','A8', '2001-06-05', 'Beta Automotores'),
    ('Audi','R8', '2019-10-05', 'Beta Automotores'),
    ('Audi','Q8', '2001-06-05', 'Sara'),
    ('BMW','Coupé', '2001-06-05', 'Beta Automotores'),
    ('BMW','Cabriolet', '2001-06-05', 'Sara'),
    ('BMW','Sedan', '2001-06-05', 'Beta Automotores'),
    ('BMW','X4', '2001-06-05', 'Beta Automotores'),
    ('Fiat','Uno', '2001-06-05', 'Sara'),
    ('Fiat','500', '2019-06-05', 'DI CHIARA CUMMINS'),
    ('Fiat','Toro', '2001-06-05', 'Sara'),
    ('Tesla','S', '2001-06-05', 'Sara'),
    ('Tesla','E', '2019-06-05', 'DI CHIARA CUMMINS'),
    ('Tesla','X', '2001-06-05', 'DI CHIARA CUMMINS'),
    ('Tesla','Y', '2019-06-05', 'DI CHIARA CUMMINS'),
    ('Audi','A7', '2001-06-05', 'Sara'),
    ('Audi','A8','2001-06-05', 'DI CHIARA CUMMINS'),
    ('Audi','R8', '2001-06-05', 'DI CHIARA CUMMINS'),
    ('Audi','Q8', '2001-06-05', 'Sara'),
    ('BMW','Coupé', '2019-06-05', 'Beta Automotores'),
    ('BMW','Cabriolet', '2019-06-05', 'Sara'),
    ('BMW','Sedan', '2019-06-05', 'Beta Automotores'),
    ('BMW','X4', '2019-06-05', 'Beta Automotores'),
]

modeloRDD = sc.parallelize(modelo)
ventasRDD = sc.parallelize(ventas)

In [116]:
modelosArg = modeloRDD.filter(lambda x: x[4]=="Argentina").map(lambda x: ((x[1],x[0]),x[4]))

ventas = ventasRDD.map(lambda x: ((x[1],x[0]),x[2]))

ventasArg = ventas.join(modelosArg)

ventasArg = ventasArg.map(lambda x: ((x[0][0],x[0][1]),(x[1][0],x[1][0],1))).reduceByKey(lambda x,y:(x[0] if x[0]>y[0] else y[0],x[1] if x[1]<y[1] else y[1], x[2]+y[2]))
ventasArg = ventasArg.filter(lambda x: x[1][0]<"2018-11-17")
ventasArg = ventasArg.sortBy(lambda x: x[1][0])

ventasArg.collect()

[(('A8', 'Audi'), ('2002-06-05', '2001-06-05', 3)),
 (('Cruze', 'Chevrolet'), ('2011-05-05', '2011-05-05', 1)),
 (('X', 'Tesla'), ('2018-06-05', '2001-06-05', 3)),
 (('Q8', 'Audi'), ('2018-06-05', '2001-06-05', 3))]

Se tiene información diaria de la cotización de acciones en el NYSE en el archivo nyse_daily.csv en el siguiente formato (symbol, date, open, measure_midday, measure_afternoon, close, volume). Por cada acción cuyo nombre está indicado en el campo symbol, tendremos una entrada por fecha con los valores open, measure_midday, measure_afternoon, y close indicando respectivamentea qué valor abrio la acción, cuál fue el valor que tuvo al medio día, cual fue su valor que tuvo por la tarde y cual fue su valor al cierre del mercado. Asi mismo en volume se indica el volumen operado ese día para esa acción.
Por otro lado se cuenta con el archivo s&p500.csv de formato(symbol, company_name) que indica aquellas acciones que deben ser consideradas para calcular el índice Standard & Poor’s 500 (S&P 500).

Se pide calcular el valor diario del índice S&P500, teniendo en cuenta que el mismo se calcula como el promedio del valor promedio de las mediciones que tuvo cada acción ese día (open, measure_midday, measure_afternoon, close), para las 500 acciones que se encuentran en el archivo s&p500.csv. El resultado debe estar en un data frame de la forma(date, index_name, value). Por ejemplo, una entrada del mismo sería(‘2019-03-24’, ‘SP500’, ‘35.46’).

*Nota: A los efectos prácticos del ejercicio consideraremos como estadísticamente significativo calcular el promedio con esas pocas mediciones.*

In [194]:
df_10_nyse = pd.DataFrame(columns = ['symbol', 'date', 'open', 'measure_midday', 'measure_afternoon', 'close', 'volume'])
df_10_sp500 = pd.DataFrame(columns = ['symbol', 'company_name'])

start_10 = pd.to_datetime('2019-1-1')
end_10 = pd.to_datetime('2019-9-11')

for _ in range(50): # En realidad, segun ejercicio son 500 !
    fecha = random_dates(start_10, end_10, 1)[0]
    fecha = pd.Timestamp(fecha.year, fecha.month, fecha.day)
    df_10_nyse = df_10_nyse.append({
                    'symbol' : random.randint(1,10), 
                    'date' : fecha, #random_dates(start_10, end_10, 1)[0], 
                    'open' : round(random.uniform(10,50),2), 
                    'measure_midday' : round(random.uniform(10,50),2), 
                    'measure_afternoon' : round(random.uniform(10,50),2), 
                    'close' : round(random.uniform(10,50),2), 
                    'volume' : random.randint(100,1000)
                }, ignore_index = True)
for i in range(10):
    df_10_sp500 = df_10_sp500.append({
                    'symbol' : i + 1, 
                    'company_name' : 'nombre {}'.format(i + 1)
                }, ignore_index = True)
df_10_nyse['volume'] = pd.to_numeric(df_10_nyse['volume'])

In [198]:

from collections import OrderedDict

def funcion(df):
    # lo usamos para preservar el orden de insercion
    data = OrderedDict()
    
    data['promedio']=float((df['open']+df['measure_midday']+df['measure_afternoon']+df['close'])/4)
    return pd.Series(data)



In [201]:
aCalcul = df_10_sp500.merge(df_10_nyse, on='symbol', how='inner')
aCalcul2 = aCalcul.groupby(['date','symbol']).apply(funcion).reset_index()
aCalcul2 = aCalcul2.groupby('date')['promedio'].agg('mean').reset_index()
aCalcul2.insert(1, "index_name", 'SP500')
aCalcul2

,date,index_name,promedio
0,2019-01-01,SP500,30.91875
1,2019-01-09,SP500,26.25250
2,2019-01-13,SP500,33.98500
3,2019-01-21,SP500,28.25250
4,2019-01-27,SP500,33.35500
5,2019-01-29,SP500,31.23750
6,2019-02-07,SP500,31.18000
7,2019-02-08,SP500,32.49500
8,2019-02-12,SP500,32.55500
9,2019-02-16,SP500,20.09000


In [4]:
#id_usuario, nick pais fecha_alta 
tags = [
    (20,"Mario Kart"),
    (2,"Mario"),
    (31,"gon"),
    (14,"pan"),
    (2,"win"),
    (6,"nop")
]
#id_publicacion , id_usuario, id_foto, comentario, fecha
canciones = [
    ('2019-11-18',1, 15, "Whake Me up","Green Day"),
    ('2019-11-18',2, 15, "Whake Me up","Green Day"),
    ('2019-11-18',3, 15, "Whake Me up","Green Day"),
    ('2019-11-17',4, 15, "Whake Me up","Green Day"),
    ('2019-11-18',5, 15, "Whake Me up","Green Day"),
    ('2019-11-15',5, 13, "Levels","Avici"),
    ('2019-11-18',2, 13, "Levels","Avici"),
    ('2019-11-15',1, 13, "Levels","Avici"),
    ('2019-11-18',1, 13, "Levels","Avici"),
    ('2019-11-17',3, 13, "Levels","Avici")
]

canciones = sc.parallelize(canciones)
tags = sc.parallelize(tags)

In [9]:
from datetime import datetime  
from datetime import timedelta  

In [32]:
nextDay('2019-11-18')

AttributeError: 'datetime.datetime' object has no attribute 'toString'

In [33]:
def nextDay(x):
    
    date = datetime.strptime(x, '%Y-%m-%d')
    nex = date + timedelta(days=1)
    
    return nex.strftime("%Y-%m-%d")

In [56]:
canciones.filter(lambda x: x[0]>='2019-11-17')\
    .map(lambda x: ((x[0],x[2]),1))\
    .reduceByKey(lambda x,y: x+y)\
    .flatMap(lambda x: [((x[0][0],x[0][1]),x[1]),((nextDay(x[0][0]),x[0][1]),-x[1])])\
    .reduceByKey(lambda x,y: x+y)\
    .filter(lambda x: ((x[1]>0) & (x[0][0]=='2019-11-18')))\
    .map(lambda x: (x[0][1],'rising'))\
    .union(tags)\
    .collect()

[(13, 'rising'),
 (15, 'rising'),
 (20, 'Mario Kart'),
 (2, 'Mario'),
 (31, 'gon'),
 (14, 'pan'),
 (2, 'win'),
 (6, 'nop')]